In [1]:
import os

In [2]:
%pwd

'/Users/kanayojustice/Documents/Data_scientist_projects/End-to-End-income-prediction-ML-project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/kanayojustice/Documents/Data_scientist_projects/End-to-End-income-prediction-ML-project'

In [5]:
#Update the entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    Local_data_file: Path
    unzip_dir: Path

In [6]:
#Update the configuration manager in src config
from src.constants import *
from incomepredictor.src.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            Local_data_file=config.Local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
#update components
import os
import urllib.request as request
import zipfile
from src.logging import logger
from src.utils.common import get_size


In [9]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.Local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.Local_data_file
            )
            logger.info(f"{filename} Download! with the following info: \n{headers}")
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.Local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.Local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [10]:
#pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-05-14 23:53:05,968: INFO:common: yaml file: /Users/kanayojustice/Documents/Data_scientist_projects/End-to-End-income-prediction-ML-project/config/config.yaml loaded successfully]
[2024-05-14 23:53:05,971: INFO:common: yaml file: /Users/kanayojustice/Documents/Data_scientist_projects/End-to-End-income-prediction-ML-project/params.yaml loaded successfully]
[2024-05-14 23:53:05,998: INFO:common: yaml file: /Users/kanayojustice/Documents/Data_scientist_projects/End-to-End-income-prediction-ML-project/schema.yaml loaded successfully]
[2024-05-14 23:53:06,001: INFO:common: created directory at: artifacts]
[2024-05-14 23:53:06,004: INFO:common: created directory at: artifacts/data_ingestion]
[2024-05-14 23:53:07,430: INFO:1730274745: artifacts/data_ingestion/income_cleaned_data.zip Download! with the following info: 
Connection: close
Content-Length: 726629
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: applicatio